In [41]:
import ee
import geemap
import os
import dotenv
import base64
import textwrap
from PIL import Image
import numpy as np
from openai import OpenAI
import requests
import h5py
import os
import folium
from IPython.display import display
from shapely.geometry import Point

# Code Relevant to Google Earth Engine

In [93]:
# Google Earth Engine
ee.Authenticate()
ee.Initialize(project='iconic-guard-454717-v9')

# Get Data from Google Earth Engine
def getSatImageryGEE(c1, c2):
    location = ee.Geometry.Point([c2 , c1])
    region = location.buffer(7000) 
    dataset = 'COPERNICUS/S2_SR_HARMONIZED'
    image = ee.ImageCollection(dataset) \
                .filterBounds(region) \
                .filterDate('2023-06-01', '2024-06-30') \
                .sort('CLOUDY_PIXEL_PERCENTAGE') \
                .first() \
                .select(['B4', 'B3', 'B2'])  # RGB bands

    return image, region

# Download the Data parsed from GEE
def downloadSatIGEE(c1, c2):
    image, region = getSatImageryGEE(c1, c2)
    geemap.ee_export_image(
        image, filename='satimagery/sentinel_rgb.tif', scale=10, region=region
        # Scale = 10m
    )

# Convert tif -> jpg
def GEE_to_brightJPG():
    img = Image.open("satimagery/sentinel_rgb.tif")

    # Convert to RGB and normalize brightness
    img = img.convert("RGB")

    # Enhance brightness slightly (optional)
    img = Image.eval(img, lambda x: x * 50)  # brighten by factor of 2

    # Save as JPEG
    img.save("satimagery/sentinel_rgb.jpg", "JPEG")
    print("Image Converted to JPG.")

# Encode the image because apparently gpt can't accept it otherwise
def prepareJPG_forOpenAI():
    with open("satimagery/sentinel_rgb.jpg", "rb") as f:
        base64_image = base64.b64encode(f.read()).decode("utf-8")

    # Format for OpenAI model
    image_content = {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
        }
    }
    return image_content

# Download + Convert + PrepForOpenAI
def getGEEImagery(c1, c2):
    downloadSatIGEE(c1, c2)
    GEE_to_brightJPG()
    ex = prepareJPG_forOpenAI()
    
    print("Image Prepared for OpenAI export.")
    return ex

# Code Relevant to OpenAI

In [163]:
# OpenAI
dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://models.github.ai/inference"

model = "openai/gpt-4.1" # or "openai/gpt-4o"
client = OpenAI()

# Sending Prompt + Image to OpenAI Model
def promptModel(p, i):
    modeloutput = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You're a geospatial analyst."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": p},
                    i
                ]
            }
        ],
        temperature=1.0,
        top_p=1.0,
        model=model
    )
    return modeloutput

# Sending Prompt to OpenAI Model
def promptONLY(p):
    modeloutput = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You're a geospatial analyst."},
            {"role": "user", "content": p}
        ],
        temperature=1.0,
        top_p=1.0,
    )
    return modeloutput


# Printing Data
def printoutput(r):
    print("\n")
    print(f"OpenAI Model: {model}")
    print(f"Dataset ID: {dataset} \n")
    print("Model Response:")
    content = r.choices[0].message.content
    sections = content.split('*')
    for section in sections:
        print(section.strip())


# Prompt OpenAI Model + Attach Image
def promptGPT(c1, c2, p, option):
    if option == 1:
        imagePreped = getGEEImagery(c1, c2)
        return promptModel(p, imagePreped)
    elif option == 2:
        return promptONLY(p)

# External DataSets I found
appEEARS_AmazonRainforest = "https://appeears.earthdatacloud.nasa.gov/download/1a6f2238-d420-498e-9bd7-6340ed3268ed"

In [94]:
X = -3.314660
Y = -61.355873

A = getGEEImagery(X, Y)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\shaaf\OneDrive\Desktop\Shaaf Media\Coding\GitHub Files\OpenAI-to-Z-challenge\satimagery\sentinel_rgb.tif
Image Converted to JPG.
Image Prepared for OpenAI export.


In [125]:
# Coordinates for Manaus, Brazil (Amazon Rainforest)

prompt = "Describe surface features in plain English based on this satellite image."

printoutput(promptGPT(X, Y, prompt, 1))

Generating URL ...
Please wait ...
Data downloaded to c:\Users\shaaf\OneDrive\Desktop\Shaaf Media\Coding\GitHub Files\OpenAI-to-Z-challenge\satimagery\sentinel_rgb.tif
Image Converted to JPG.
Image Prepared for OpenAI export.


OpenAI Model: openai/gpt-4.1
Dataset ID: COPERNICUS/S2_SR_HARMONIZED 

Model Response:
This satellite image shows a large, mostly continuous area of dense green
vegetation, likely representing a forest or jungle. There are small,
irregularly-shaped lighter green and brown patches scattered throughout, which
are likely clearings or areas where vegetation has been removed, possibly for
agriculture or small settlements. Toward the right side of the image, there are
areas with more visible variation in color and texture, indicating uneven
terrain, possibly hills or ridges. There are also a few linear features that
could be roads or tracks cutting through the forest. Overall, the landscape
appears to be natural and heavily forested, with some small signs of human
act

In [108]:
# We now begin visualizing the second Dataset

from collections import Counter
import rasterio

with rasterio.open("AppEEARS/MCD12Q1.061_LC_Prop2_doy2020001_aid0001.tif") as src:
    data = src.read(1)

counts = Counter(data.flatten())
print(counts)



Counter({10: 3229248, 255: 1422346, 3: 69843, 20: 16631, 30: 11977, 9: 563, 40: 200, 1: 4})


In [111]:
# Readable Text
modis_pft_classes = {
    1: "Evergreen Needleleaf Trees",
    3: "Deciduous Needleleaf Trees",
    9: "Flooded Vegetation",
    10: "Urban Areas",
    20: "Unknown",
    30: "Unknown",
    40: "Unknown",
    255: "No Data"
}

summary = []
for code, count in counts.items():
    label = modis_pft_classes.get(code, "Unknown")
    summary.append(f"{label} (Class {code}): {count} pixels")

report_text = "\n".join(summary)
print(report_text)


No Data (Class 255): 1422346 pixels
Urban Areas (Class 10): 3229248 pixels
Deciduous Needleleaf Trees (Class 3): 69843 pixels
Unknown (Class 30): 11977 pixels
Unknown (Class 20): 16631 pixels
Unknown (Class 40): 200 pixels
Flooded Vegetation (Class 9): 563 pixels
Evergreen Needleleaf Trees (Class 1): 4 pixels


In [166]:
prompt2 = f"""You are a geospatial analyst. Given the following MODIS land cover pixel counts (PFT classes), create a **concise, well-formatted markdown report** with:

- A clean summary table with headers
- Bullet-pointed key insights
- A short paragraph interpretation (add a * after every 10 words)
- Add in a * everytime a new line starts


Here is the Data: 
{report_text}
"""

printoutput(promptGPT(0, 0, prompt2, 2))



OpenAI Model: openai/gpt-4.1
Dataset ID: COPERNICUS/S2_SR_HARMONIZED 

Model Response:

# MODIS Land Cover Pixel Counts Summary
| Land Cover Class                     | Class Code | Pixel Count |
|--------------------------------------|------------|-------------|
| No Data                              | 255        | 1,422,346   |
| Urban Areas                          | 10         | 3,229,248   |
| Deciduous Needleleaf Trees           | 3          | 69,843      |
| Unknown                              | 30         | 11,977      |
| Unknown                              | 20         | 16,631      |
| Unknown                              | 40         | 200         |
| Flooded Vegetation                   | 9          | 563         |
| Evergreen Needleleaf Trees           | 1          | 4           |
## Key Insights
- Urban Areas (Class 10) overwhelmingly dominate the pixel
count in this dataset.

- More than 1.4 million pixels are marked as No Data, indicating substantial data gaps.

- 